A system of three linear equations has the following expression in matrix notation:
$$
\mathbf{y} = \mathbf{W}\mathbf{x}
$$

with
* $\mathbf{x}$ is a column vector representing our inputs whose size is (3,1).
* $\mathbf{y}$ is a column vector representing our outputs whose size is (3,1).
* $\mathbf{W}$ is a matrix representing our coefficients whose size is (3,3).

# Experiment (with ideal characteristics)
Let $X = \begin{bmatrix} 9. \\ 15. \\ 35. \end{bmatrix}$ and $W = \begin{bmatrix} 8. & 3. & -2. \\ -4. & 7. & 5. \\ 3. & 4. & -12. \end{bmatrix}$.

## Approach 1
We construct a single perceptron deep learning model with only a linear activation. No backward propagation is performed.

Once the model is implemented and mapped to the corresponding memristive deep neural network (MDNN), we feed the input vector $X$ to the corresponding memristive network.

In [7]:
import copy
from memtorch.mn.Module import patch_model
from memtorch.map.Parameter import naive_map
from memtorch.map.Input import naive_scale

device = torch.device('cpu' if 'cpu' in memtorch.__version__ else 'cuda')

reference_memristor = memtorch.bh.memristor.VTEAM
reference_memristor_params = {"time_series_resolution": 1e-10, "r_off": 100e3, "r_on": 10e3,}
model = Net().to(device)

patched_model = patch_model(copy.deepcopy(model),
                          memristor_model=reference_memristor,
                          memristor_model_params=reference_memristor_params,
                          module_parameters_to_patch=[torch.nn.Conv2d],
                          mapping_routine=naive_map,
                          transistor=True,
                          programming_routine=None,
                          tile_shape=(128, 128),
                          max_input_voltage=0.3,
                          scaling_routine=naive_scale,
                          ADC_resolution=8,
                          ADC_overflow_rate=0.,
                          quant_method='linear')
patched_model.tune_()
patched_model.forward(torch.tensor(X, requires_grad=True))

NameError: name 'Net' is not defined

## Approach 2
We construct a memristor crossbar array. 

In [12]:
import torch
import torch.nn as nn
import numpy as np
import memtorch

from memtorch.map.Parameter import naive_map
from memtorch.bh.crossbar.Crossbar import init_crossbar

class IdealMCA():
    def __init__(self, W):
        super(IdealMCA, self).__init__()
        # self.X = torch.tensor(X, requires_grad= True)
        self.W = torch.tensor(W, requires_grad= True)
        self.define_memristor_behaviours()
        self.define_crossbar()
        pass
    
    def define_memristor_behaviours(self):
        self.reference_memristor = memtorch.bh.memristor.VTEAM
        self.reference_memristor_params = {"time_series_resolution": 1e-10, "r_off": 100e3, "r_on": 10e3,}
        pass

    def define_crossbar(self):
        self.crossbars, self.operation = init_crossbar(
            weights = self.W,
            memristor_model= self.reference_memristor,
            memristor_model_params= self.reference_memristor_params,
            transistor= True,
            mapping_routine= naive_map,
            programming_routine= None,
            tile_shape= (3,3),
            scheme= memtorch.bh.crossbar.Scheme.DoubleColumn,
        )
    
    def forward(self,X):
        self.X = torch.tensor(X, requires_grad= True)
        #self.W = torch.tensor(W, requires_grad= True)
        self.W = self.operation(self.crossbars, lambda crossbar: crossbar.conductance_matrix)
        self.results = torch.matmul(self.W, self.X)
        print(f'Crossbar Product: {self.results}')
        return self.results 
        pass



In [16]:
W = [[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]]

test_model = IdealMCA(W)

In [19]:
X = [100., 100., 1.]

test_results = test_model.forward(X)

Crossbar Product: tensor([[0.0090, 0.0090, 0.0090]], grad_fn=<UnsafeViewBackward0>)


The reason this produces a scaled output is that the elements of $W$ are encoded on the crossbar using conductance values which can be represented by each device, i.e., they are scaled and encoded between $1/r_{off}$ and $1/r_{on}$. In MCA, the Ohm's law is exploited to perform Vector-Matrix Multiplication, so:
* Elements of tensor $X$ are encoded as Word Line (WL) voltages.
* Elements of tensor $Y$ is given as: $I=GV$ (assuming linear I/V characteristics and that line and source resistances are negligible/select devices are used). Since encoded elements of tensor $X$ are scaled, the resulting currents, i.e., $I$, must be scaled much that they are approximately equal to the desired product, which is $GV \equiv WX$.


## Experiment (with non-ideal characteristics)